In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [ ]:
!pip install transformers newspaper3k lxml_html_clean googlesearch-python yfinance ultralytics opencv-python

In [ ]:
!pip install gradio

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [14]:
from google.colab import files
uploaded = files.upload()  # Choose your image


Saving Screenshot (90).png to Screenshot (90).png


In [15]:
import math
import yfinance as yf
from newspaper import Article
from googlesearch import search
from transformers import pipeline
from ultralytics import YOLO

# ------------------- LLM Research Agent -------------------
class LLMResearchAgent:
    def __init__(self, model_name="tiiuae/falcon-7b-instruct", device=0):
        # Fully open-source, no login required
        print("Loading Falcon-7B model (this may take a while)...")
        self.generator = pipeline("text-generation", model=model_name, device=device)

    def summarize(self, text):
        if not text.strip():
            return "No text to summarize."
        output = self.generator(f"Summarize this:\n{text}", max_length=512, do_sample=True)
        return output[0]["generated_text"]

# ------------------- Web Agent -------------------
class WebAgent:
    def search(self, query, max_results=5):
        try:
            return list(search(query, num_results=max_results))
        except:
            return ["Web search failed"]

# ------------------- News Agent -------------------
class NewsAgent:
    def read_article(self, url, max_chars=1000):
        try:
            article = Article(url)
            article.download()
            article.parse()
            return article.text[:max_chars]
        except:
            return None

# ------------------- Calculator Agent -------------------
class CalculatorAgent:
    def calculate(self, expression):
        try:
            return eval(expression, {"__builtins__": {}}, math.__dict__)
        except Exception as e:
            return str(e)

# ------------------- Finance Agent -------------------
class FinanceAgent:
    def get_stock_info(self, ticker):
        try:
            stock = yf.Ticker(ticker)
            info = stock.info
            return {
                "name": info.get("shortName"),
                "price": info.get("currentPrice"),
                "sector": info.get("sector"),
                "marketCap": info.get("marketCap")
            }
        except:
            return {"error": "Failed to fetch stock info"}

# ------------------- Vision Agent -------------------
class VisionAgent:
    def __init__(self, model_path="yolov8n.pt"):
        # Use uploaded file or default YOLO model
        self.model = YOLO(model_path)

    def detect_objects(self, image_path):
        try:
            results = self.model(image_path)
            return results.pandas().xyxy[0]  # Pandas DataFrame
        except:
            return "Object detection failed"

# ------------------- Team Agent -------------------
class AgentTeam:
    def __init__(self):
        self.research_agent = LLMResearchAgent()
        self.web_agent = WebAgent()
        self.news_agent = NewsAgent()
        self.calculator_agent = CalculatorAgent()
        self.finance_agent = FinanceAgent()
        self.vision_agent = VisionAgent()

    def run(self, query):
        query_lower = query.lower()
        # Routing logic
        if any(op in query_lower for op in ["+", "-", "*", "/", "factorial", "sqrt"]):
            return self.calculator_agent.calculate(query)
        elif any(tick in query_lower.upper() for tick in ["AAPL","TSLA","GOOG","MSFT"]):
            ticker = query.split()[-1].upper()
            return self.finance_agent.get_stock_info(ticker)
        elif "image" in query_lower or "detect" in query_lower:
            # Use first uploaded image
            image_path = list(uploaded.keys())[0]
            return self.vision_agent.detect_objects(image_path)
        else:
            urls = self.web_agent.search(query)
            articles_text = "\n".join([self.news_agent.read_article(url) for url in urls if self.news_agent.read_article(url)])
            return self.research_agent.summarize(articles_text)


In [17]:
team = AgentTeam()

# --- Research Example ---
query = "Tesla stock news"
print("=== RESEARCH SUMMARY ===")
print(team.run(query))

# --- Calculator Example ---
query2 = "factorial(5) + sqrt(16)"
print("=== CALCULATOR RESULT ===")
print(team.run(query2))

# --- Finance Example ---
query3 = "Stock info TSLA"
print("=== FINANCE INFO ===")
print(team.run(query3))

# --- Vision Example ---
query4 = "Detect objects in image"
print("=== OBJECT DETECTION ===")
print(team.run(query4))


Loading Falcon-7B model (this may take a while)...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cpu


=== RESEARCH SUMMARY ===
No text to summarize.
=== CALCULATOR RESULT ===
124.0
=== FINANCE INFO ===
{'name': 'Tesla, Inc.', 'price': 451.45, 'sector': 'Consumer Cyclical', 'marketCap': 1501441032192}
=== OBJECT DETECTION ===

image 1/1 /content/Screenshot (90).png: 384x640 1 person, 1 cell phone, 375.7ms
Speed: 17.3ms preprocess, 375.7ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)
Object detection failed


In [18]:
import gradio as gr

def assistant(query):
    return team.run(query)

gr.Interface(fn=assistant, inputs="text", outputs="text", title="Open-Source Multi-Agent AI").launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5ad79cb6b27bb56dee.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
